# Aufgabe - Spielbank

Passend zu dem zugehörigen Abschnitt, soll eine Spielbank objektorientiert implementiert werden. Dabei besteht diese Aufgabe aus zwei Teilen. Zuerst wird ein Automat erstellt, an dem die Kund:innen spielen können. Daraufhin soll die Verwaltung von Kunden:innen und Mitarbeiter:innen realisiert werden. Beide Gruppen sollen jedoch unterschiedliche Zugriffe auf einen Automaten besitzen.  
Für die Tests dieser Aufgabe ist es wichtig, dass Sie das Notebook im ganzen ausführen. Verwenden Sie dafür dafür beispielsweise unter *Kernel* den Punkt *Restart*.<br />
Wählen Sie passende Variablentypen und Sichtbarkeiten.

## Vorbereitungen
Bevor die Klasse `Automat` implementiert werden kann, müssen einige Vorbereitungen getroffen werden. Da es sich in diesem Fall um eine klassische Slotmachine handelt, werden Symbole gebraucht, die in einem 2D-Display angezeigt werden können.  
Implementieren Sie dafür eine Enumeration `Symbol`, die die Werte `Kreuz`, `Apfel`, `Banane`, `Melone`, `Zitrone` enthält. Während `Kreuz` nur als Platzhalter für den Start des Automatens dienen soll, werden die anderen Symbole nach dem betätigen des Spielbuttons zufällig ausgewählt. Schreiben Sie dafür eine Methode `gibSymbole()`, die eine Liste mit allen für das Spiel wichtige *Bilder* enthält.

In [1]:
// BEGIN SOLUTION
enum class Symbol{
    Kreuz, Apfel, Banane, Melone, Zitrone;
    
    fun gibSymbole() = listOf(Apfel, Banane, Melone, Zitrone)
}
// END SOLUTION

In [2]:
val s1 = Symbol.Kreuz
val s2 = Symbol.Apfel
val s3 = Symbol.Banane
val s4 = Symbol.Melone
val s5 = Symbol.Zitrone
if(Symbol.Kreuz.gibSymbole() != listOf(Symbol.Apfel, Symbol.Banane, Symbol.Melone, Symbol.Zitrone))
    throw AssertionError("Symbol.Kreuz.gibSymbole(): ${Symbol.Kreuz.gibSymbole()} != ${listOf(Symbol.Apfel, Symbol.Banane, Symbol.Melone, Symbol.Zitrone)}")

Im nächsten Schritt sollen zwei Schnittstellen für den Automaten erstellt werden.  
Die `Kundensicht` soll eine ganzzahlige `automatennummer` und eine Methode `spielen()` vorschreiben. Als Parameter soll die Methode den Einsatz als Gleitkommazahl erhalten und zum Schluss den Gewinn, ebenfalls als Gleitkommazahl, zurückgeben.  
Für die Mitarbeiter:innen soll eine weitere Schnittstelle mit dem Namen `Mitarbeitersicht` angegeben werden. Es soll ebenfalls eine `automatennummer` vorhanden sein. Ergänzt wird sie um zwei Methoden `quoteSetzen`, die einen Faktor des Datentyps `Double` übergeben bekommt. Dieser soll die Höhe des Gewinns verändert. Mit `gibStatistiken()` soll der Mitarbeiter:in Zugriff auf die Daten des Automatens erlangen. Die Rückgabe stellt eine Liste aus Gleitkommazahlen dar.

In [3]:
// BEGIN SOLUTION
interface Kundensicht{
    val automatennummer: Int
    
    fun spielen(einsatz: Double): Double
}

interface Mitarbeitersicht{
    val automatennummer: Int
    
    fun quoteSetzen(neueQuote: Double)
    fun gibStatistiken(): List<Double>
}
// END SOLUTION

In [4]:
class TestAutomat(): Kundensicht, Mitarbeitersicht{
    override val automatennummer = 0
    override fun spielen(einsatz: Double) = 2.0
    override fun quoteSetzen(neueQuote: Double) {
        println("Test")
    }
    override fun gibStatistiken() = listOf(0.00)
}

## Automat
Jetzt soll die Klasse `Automat` implementiert werden. Sie soll sowohl die `Kundensicht` als auch die `Mitarbeitersicht` enthalten.  
- Die `automatennummer` soll für jeden Automaten einzigartig sein und von 0 aufsteigend vergeben werden.  
- In dem Feld `anzeige` soll ein 2-dimensionales Array der Größe 4x4 gespeichert werden, das Symbole enthält. Bei Erzeugung eines Automatens soll die Anzeige nur aus Kreuzen bestehen.  
- `ausgegeben` speichert die Summe der bisherig ausgeschütteten Gewinne und `eingeworfen` den gesamten Einsatz.
- Die `quote` soll eine Gleitkommazahl sein, die die Höhe des Gewinns maßgeblich verändert und von einer Mitarbeiter:in mit der Methode `quoteSetzen()` angepasst werden kann. Zu Beginn soll die Quote *2.0* sein.  
- Der Wahrhietswert `besetzt` speichter, ob ein Kunde an dem Automaten sitzt (`true`) oder dieser frei ist (`false`). Dieses Feld soll öffentlich sein, da jede Spielbankenbesucher:in sieht, ob eine Kund:in an dem Gerät sitzt.  
- Die von `Mitarbeitersicht` vorgegebene Methode `gibStatistiken()`, gibt die Felder `eingeworfen`, `ausgegeben` und `quote` in einer Liste zurück.  
- Spielt eine Kund:in an dem Automaten wird zuerst der Einsatz zum Feld `eingeworfen` addiert. Daraufhin wird die Anzeige aktualisiert, indem jedem Element zufällig eines der Symbole zugewiesen wird. Zum Schluss wird der Gewinn zu der zufälligen Anzeige berechnet und zurückgegeben.  
- Der Gewinn soll in der Methode `berechneGewinn()` ermittelt werden. Als Parameter werden ihr der Einsatz sowie eine Anzeige übergeben. Zur Vereinfachung sollen die Zeilen und Spalten gezählt werden, in denen nur Symbole einer Art zu finden sind. Zusätzlich sollen auch die Diagonalen getestet werden. Zum Schluss wird der Gewinn statistisch erfasst und zurückgegeben. Die Höhe des Gewinns ergibt sich aus der Formel:
>(anzahlZeilen + anzahlSpalten + 5*anzahlDiagonalen) * quote * einsatz

In [5]:
// BEGIN SOLUTION
class Automat(): Kundensicht, Mitarbeitersicht{
    
    companion object{
        var zaehler = 0
    }
    override val automatennummer = zaehler++
    
    private val anzeige = arrayOf(
        arrayOf(Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz),
        arrayOf(Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz),
        arrayOf(Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz),
        arrayOf(Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz, Symbol.Kreuz)
    )
    
    private var ausgegeben = 0.00
    private var eingeworfen = 0.00
    private var quote = 2.00
    public var besetzt = false
    
    override fun quoteSetzen(neueQuote: Double){
        quote = neueQuote
    }
    
    override fun gibStatistiken() = listOf(eingeworfen, ausgegeben, quote)
    
    override fun spielen(einsatz: Double): Double{
        eingeworfen += einsatz
        anzeigeAktualisieren()
        return berechneGewinn(einsatz, anzeige)
    }
    
    private fun anzeigeAktualisieren(){
        for(i in 0 until anzeige.size){
            for(j in 0 until anzeige[i].size){
                anzeige[i][j] = Symbol.Kreuz.gibSymbole().random()
            }
        }
    }
    
    fun berechneGewinn(einsatz: Double, anzeige: Array<Array<Symbol>>): Double{
        var zeilen = 0
        var spalten = 0
        var diagonalen = 0
        
        for(i in 0 until anzeige.size){
            if(anzeige[i].toSet().size == 1){
                zeilen++
            }
            if(setOf(anzeige[0][i], anzeige[1][i], anzeige[2][i], anzeige[3][i]).size == 1){
                spalten++
            }
        }
        if(setOf(anzeige[0][0], anzeige[1][1], anzeige[2][2], anzeige[3][3]).size == 1){
            diagonalen++
        }
        if(setOf(anzeige[3][0], anzeige[2][1], anzeige[1][2], anzeige[0][3]).size == 1){
            diagonalen++
        }
        
        val gewinn = (zeilen + spalten + 5*diagonalen) * quote * einsatz
        ausgegeben += gewinn
        return gewinn
    }
    
    //Optional
    fun anzeigeAusgeben(){
        anzeige.forEach { it.forEach { print("$it ") }; println() }
    }
}
// END SOLUTION

In [6]:
val a1 = Automat()
val a2 = Automat()

//automantennummer
if(a1.automatennummer != 0 && a2.automatennummer != 1)
    throw AssertionError("a1.automatennummer: ${a1.automatennummer} != 0, a2.automatennummer: ${a2.automatennummer} != 1")

//besetzt
if(a1.besetzt)
    throw AssertionError("a1.besetzt: ${a1.besetzt} != false")
a1.besetzt = true
if(!a1.besetzt)
    throw AssertionError("a1.besetzt: ${a1.besetzt} != true")

//quoteSetzen()
a1.quoteSetzen(2.5)

//gibStatistiken()
if(a1.gibStatistiken() != listOf(0.00, 0.00, 2.50))
    throw AssertionError("a1.gibStatistiken(): ${a1.gibStatistiken()} != ${listOf(0.00, 0.00, 2.50)}")

//spielen
var gewonnen = 0.00
repeat(50){
    gewonnen += a1.spielen(1.00)
}
if(a1.gibStatistiken() != listOf(50.00, gewonnen, 2.50))
    throw AssertionError("Fehler in der Methode spielen(): a1.gibStatistiken() nach 50 Spielen: ${a1.gibStatistiken()} != ${listOf(50.00, gewonnen, 2.50)}")
    
//berechneGewinn()
val zeilenGewinn = arrayOf(
    arrayOf(Symbol.Apfel, Symbol.Apfel, Symbol.Apfel, Symbol.Apfel),
    arrayOf(Symbol.Banane, Symbol.Banane, Symbol.Banane, Symbol.Banane),
    arrayOf(Symbol.Melone, Symbol.Melone, Symbol.Melone, Symbol.Melone),
    arrayOf(Symbol.Zitrone, Symbol.Zitrone, Symbol.Zitrone, Symbol.Zitrone)
)
if(a1.berechneGewinn(12.34, zeilenGewinn) != 123.4)
    throw AssertionError("Fehler beim Berechnen des Gewinns bei richtigen Zeilen: a1.berechneGewinn(12.34, zeilenGewinn): ${a1.berechneGewinn(12.34, zeilenGewinn)} != 123.4")
    
val spaltenGewinn = arrayOf(
    arrayOf(Symbol.Apfel, Symbol.Banane, Symbol.Melone, Symbol.Zitrone),
    arrayOf(Symbol.Apfel, Symbol.Banane, Symbol.Melone, Symbol.Zitrone),
    arrayOf(Symbol.Apfel, Symbol.Banane, Symbol.Melone, Symbol.Zitrone),
    arrayOf(Symbol.Apfel, Symbol.Banane, Symbol.Melone, Symbol.Zitrone)
)
if(a1.berechneGewinn(42.43, spaltenGewinn) != 424.3)
    throw AssertionError("Fehler beim Berechnen des Gewinns bei richtigen Spalten: a1.berechneGewinn(42.42, spaltenGewinn): ${a1.berechneGewinn(42.43, spaltenGewinn)} != 424.3")
    
val diagonalenGewinn = arrayOf(
    arrayOf(Symbol.Apfel, Symbol.Banane, Symbol.Melone, Symbol.Banane),
    arrayOf(Symbol.Apfel, Symbol.Apfel, Symbol.Banane, Symbol.Zitrone),
    arrayOf(Symbol.Apfel, Symbol.Banane, Symbol.Apfel, Symbol.Zitrone),
    arrayOf(Symbol.Banane, Symbol.Banane, Symbol.Melone, Symbol.Apfel)
)
if(a1.berechneGewinn(38.53, diagonalenGewinn) != 963.25)
    throw AssertionError("Fehler beim Berechnen des Gewinns bei richtigen Diagonalen: a1.berechneGewinn(38.53, spaltenGewinn): ${a1.berechneGewinn(38.53, diagonalenGewinn)} != 963.25")

## Verwaltung von Kund:innen und Mitarbeiter:innen
Im zweiten Teil der Aufgabe sollen verschiedene Personen erfasst werden. 

### Person

Die Klassen `Kunde` und `Mitarbeiter` erben von der abstrakten Klasse `Person`, die den Namen `name` und das Alter `alter` speichert.  

### Kunde
- Zusätzlich soll eine Kund:in ein eigenes, bei Erzeugung übergebenes Budget und einen ebenfalls bei der Erzeugung zugewiesenen Automaten der Klasse `Kundensicht` besitzen.  
- In dem Feld `mitgliedsnummer` soll eine ganzzahlige, einzigartige und von 0 aufsteigende Zahl gespeichert werden, die bei der Verwaltung Verwendung finden wird.  
- Zusätzlich kann mit der Methode `anAutomatSpielen()` mit einem übergebenen Einsatz eine Runde an dem übergebenen Automaten gestartet werden. Ist der Einsatz größer als das Budget, soll ein eigens definierter Fehler `KeinBudget` geworfen werden, der mit Hilfe des Einsatzes und des Budgets eine aussagekräftige Fehlermeldung besitzt. Ansonsten wird der Einsatz vom Budget abgezogen und das Spiel am Automaten gestartet. Der Gewinn wird wiederum zu dem Budget addiert.  
- Zum Schluss soll noch eine Methode `gibAutomatennummer()`implementiert werden, die die Nummer des Automaten zurückgibt.

### Mitarbeiter
- Eine Mitarbeiter:in bekommt zusätzlich ein Array mit Automaten der Klasse `Mitarbeitersicht` übergeben.  
- Wie auch bei der Klasse `Kunde` hat ein Objekt der Klasse `Mitarbeiter` eine einzigartige und von 0 aus aufsteigende `mitarbeiternummer`.  
- Mit der Methode `aendereQuote()` soll eine Mitarbeiter:in die Quote eines bestimmten Automatens ändern können. Dazu wird die neue Quote und die Automatennummer übergeben.
- Zusätzlich soll die Statistik eines Automatens, gekennzeichnet mit seiner Nummer, bei der Methode `gibStatistik()` zurückgegeben werden.

In [7]:
// BEGIN SOLUTION
abstract class Person(val name: String, val alter: Int)

class Kunde(name: String, alter: Int, private var budget: Double, private val automat: Kundensicht): Person(name, alter){
    companion object{
        var zaehler = 0
    }
    val mitgliedsnummer = zaehler++
    
    fun anAutomatSpielen(einsatz: Double){
        if(einsatz < budget){
            budget -= einsatz
            budget += automat.spielen(einsatz)
        }
        else{
            throw KeinBudget(einsatz, budget)
        }
    }
    
    fun gibAutomatennummer() = automat.automatennummer
}

class KeinBudget(einsatz: Double, budget: Double): Exception("Der Einsatz $einsatz übersteigt das aktuelle Budget $budget")

class Mitarbeiter(name: String, alter: Int, val automaten: Array<Mitarbeitersicht>): Person(name, alter){
    companion object{
        var zaehler = 0
    }
    val mitarbeiternummer = zaehler++
    
    fun aendereQuote(automatennummer: Int, neueQuote: Double){
        automaten[automatennummer].quoteSetzen(neueQuote)
    }
    
    fun gibStatistiken(automatennummer: Int) = automaten[automatennummer].gibStatistiken()
}
// END SOLUTION

In [8]:
val a1 = Automat()
val a2 = Automat()
val a3 = Automat()

if(!Person::class.isAbstract)
    throw IllegalStateException("Die Klasse Person sollte abstrakt sein.")

//Kunde
val k1 = Kunde("Kunde1", 25, 5.00, a1)
val k2 = Kunde("Kunde2", 32, 15.00, a1)

if(k1 !is Person)
    throw IllegalStateException("Die Klasse Kunde sollte von Person erben.")

//name
if(k1.name != "Kunde1")
    throw AssertionError("k1.name: ${k1.name} != Kunde1")
    
//alter
if(k1.alter != 25)
    throw AssertionError("k1.alter: ${k1.alter} != 25")
    
//mitgliedsnummer
if(k1.mitgliedsnummer != 0 && k2.mitgliedsnummer != 1)
    throw AssertionError("k1.mitgliedsnummer: ${k1.mitgliedsnummer} != 0, k2.mitgliedsnummer: ${k2.mitgliedsnummer} != 1")

//anAutomatSpielen()
k1.anAutomatSpielen(2.5)
try{
    k1.anAutomatSpielen(15.0)
    throw UnsupportedOperationException("Es sollte der Fehler KeinBudget geworfen werden.")
}catch(e: KeinBudget){
}

//Mitarbeiter
val m1 = Mitarbeiter("Mitarbeiter1", 42, arrayOf(a1, a2, a3))
val m2 = Mitarbeiter("Mitarbeiter2", 34, arrayOf(a1, a2, a3))

if(m1 !is Person)
    throw IllegalStateException("Die Klasse Mitarbeiter sollte von Person erben.")

//mitarbeiternummer
if(m1.mitarbeiternummer != 0 && m2.mitarbeiternummer != 1)
    throw AssertionError("m1.mitarbeiternummer: ${m1.mitarbeiternummer} != 0, m2.mitarbeiternummer: ${m2.mitarbeiternummer} != 1")

    
//aendereQuote()
m1.aendereQuote(1, 1.0)
if(a2.gibStatistiken()[2] != 1.0)
    throw AssertionError("Quote von a2: ${a2.gibStatistiken()[2]} != 1.0")
    
//gibStatistiken()
if(m1.gibStatistiken(0) != a1.gibStatistiken())
    throw AssertionError("m1.gibStatistiken(0): ${m1.gibStatistiken(0)} != ${a1.gibStatistiken()}")

### Personenverwaltung

Kund:innen und Mitarbeiter:innen sollen mit Hilfe von Generizität mit der selben Klasse `Personenverwaltung` gespeichert werden können. Beschränken Sie den generischen Parameter auf Unterklassen der Klasse `Person`.  
- Die Personen sollen in einer privaten, veränderbaren Liste gespeichert werden.  
- Mit der Methode `hinzufuegen()` soll eine übergebene Person der liste angehängt werden.  
- `entfernen()` gibt einen Wahrheitswert zurück, der von dem Erfolg des Entfernens abhängt. Übergeben wird der Methode eine ganze Zahl, die entweder die `mitgliedsnummer` oder `mitarbeiternummer` darstellt. Wird keine Person mit der übergebenen Nummer in der Liste gefunden, schlägt das Entfernen fehl. Ansonsten wird sie aus der Liste gelöscht.  
- Die Möglichkeit nach einer Person in der Liste zu Suchen, soll `gibPerson()` bereitstellen. Das ausschlaggebende Kriterium ist wieder die Nummer der Person. Wird Suchergebnis gefunden, soll `null` zurückgegeben werden, ansonsten das gefundene Objekt.

In [9]:
// BEGIN SOLUTION
class Personenverwaltung<T: Person>(){
    private val liste = mutableListOf<T>()
    
    fun hinzufuegen(p: T) = liste.add(p)
    
    fun entfernen(nummer: Int): Boolean {
        val person = liste.find { 
            when(it){
                is Kunde -> it.mitgliedsnummer == nummer
                is Mitarbeiter -> it.mitarbeiternummer == nummer
                else -> return false
            }
        }
        if(person != null){
            liste.remove(person)
            return true
        }
        return false     
    }
    
    fun gibPerson(nummer: Int) = liste.find { 
        when(it){
            is Kunde -> it.mitgliedsnummer == nummer
            is Mitarbeiter -> it.mitarbeiternummer == nummer
            else -> return null
        }
    } 
}
// END SOLUTION

In [10]:
val a1 = Automat()
val k1 = Kunde("Kunde1", 25, 5.00, a1)
val k2 = Kunde("Kunde2", 35, 15.00, a1)

val kunden = Personenverwaltung<Kunde>()

//hinzufuegen()
kunden.hinzufuegen(k1)
kunden.hinzufuegen(k2)

//gibPerson()
if (kunden.gibPerson(k1.mitgliedsnummer) != k1)
    throw AssertionError("kunden.gibPerson(k1.mitgliedsnummer): ${kunden.gibPerson(k1.mitgliedsnummer)} != ${k1}")
//entfernen()
if (!kunden.entfernen(k1.mitgliedsnummer))
    throw AssertionError("kunden.entfernen(k1.mitgliedsnummer): false != true")
if (kunden.entfernen(k1.mitgliedsnummer))
    throw AssertionError("kunden.entfernen(k1.mitgliedsnummer): true != false")
if (kunden.gibPerson(k1.mitgliedsnummer) != null)
    throw AssertionError("kunden.gibPerson(k1.mitgliedsnummer): ${kunden.gibPerson(k1.mitgliedsnummer)} != null")

## Spielbank

In der Klasse `Spielbank` sollen die implementierten Klassen in Zusammenhang gestellt und eine vereinfachte Spielbank realisiert werden.
- Bei Erzeugung werden drei Parameter übergeben: der Name der Spielbank, deren Ort und die Anzahl der vorhandenen Automaten
- In dem privaten Feld `kunden` soll ein Objekt der Klasse `Personenverwaltung` gespeichert werden, die Kund:innen verwaltet. `mitarbeiter` enthält im Gegensatz die Verwaltung der Mitarbeiter:innen.
- `automaten` ist ein privates Feld, dass ein Array aus Automaten enhält. Die Anzahl der Automaten ist anhängig von dem Übergabeparameter.
- `registriereKunde()` wird der Name, das Alter und Budget übergeben. Die Rückgabe stellt ein Objekt der Klasse `Kunde` dar. Falls das Alter unter 21 ist, soll ein eigens definierte Fehler `AltersgrenzeUnterschritten` geworfen werden, der auf das Mindestalter von 21 Jahren hinweist. Besteht die Kund:in den Alterstest, wird ein freier Automat zugewiesen und das Objekt in die Kundenverwaltung hinzugefügt. Sollten alle Automaten belegt sein, soll ein Fehler der Klasse `KeinAutomatFrei` geworfen werden.
- Verlässt eine Kund:in die Spielbank, soll die Methode `entferneKunde()` mit der Nummer der Kund:in aufgerufen werden. Daraufhin wird der zugehörige Automat frei und der Eintrag in der Kundenverwaltung gelöscht.
- Analog soll mit den Mitarbeiter:innen verfahren werden. Der Methode `registriereMitarbeiter()` wird ein Name und Alter übergeben. Das Ergebnis ist ein Objekt der Klasse `Mitarbeiter`, das der Mitarbeiterverwaltung hinzugefügt wurde.
- `entferneMitarbeiter()` löscht eine Mitarbeiter:in aus der Mitarbeiterverwaltung abhängig von der Mitarbeiternummer.

In [11]:
// BEGIN SOLUTION 
class Spielbank(val name: String, val ort: String, val anzahlAutomaten: Int){
    private val kunden = Personenverwaltung<Kunde>()
    private val mitarbeiter = Personenverwaltung<Mitarbeiter>()
    private val automaten = Array(anzahlAutomaten) { Automat() }
    
    fun registriereKunde(name: String, alter: Int, budget: Double): Kunde{
        if(alter > 20){
            val automat = automaten.firstOrNull { it.besetzt == false }
            if(automat != null){
                val neuerKunde = Kunde(name, alter, budget, automat)
                kunden.hinzufuegen(neuerKunde)
                automat.besetzt = true
                return neuerKunde
            }
            else
                throw KeinAutomatFrei()
        }
        else
            throw AltersgrenzeUnterschritten()
    }
    
    fun entferneKunde(mitgliedsnummer: Int): Boolean{
        val automatennummer = kunden.gibPerson(mitgliedsnummer)?.gibAutomatennummer()
        val automat = automaten.find { it.automatennummer == automatennummer }
        automat?.besetzt == false
        return kunden.entfernen(mitgliedsnummer)
    }
    
    fun registriereMitarbeiter(name: String, alter: Int): Mitarbeiter{
        val neuerMitarbeiter = Mitarbeiter(name, alter, automaten as Array<Mitarbeitersicht>)
        mitarbeiter.hinzufuegen(neuerMitarbeiter)
        return neuerMitarbeiter
    }
    
    fun entferneMitarbeiter(mitarbeiternummer: Int) = mitarbeiter.entfernen(mitarbeiternummer)
    
}

class AltersgrenzeUnterschritten(): Exception("Für einen Besuch einer Spielbank müssen Sie 21 Jahre alt sein.")
class KeinAutomatFrei(): Exception("Es sind alle Automaten belegt.")
// END SOLUTION

In [12]:
val spielbank = Spielbank("Spialbank Bad Steben", "Bad Steben", 3)

//registriereKunde()
val k1 = spielbank.registriereKunde("Max Mustermann", 23, 10.00)
if (k1.mitgliedsnummer != 4)
    throw AssertionError("spielbank.registriereKunde(Max Mustermann, 23, 10.00).mitgliedsnummer: ${k1.mitgliedsnummer} != 4")

//Fehler AltersgrenzeUnterschritten
try{
    val k2 = spielbank.registriereKunde("Erika Mustermann", 18,5.00)
    throw UnsupportedOperationException("Es sollte der Fehler AltersgrenzeUnterschritten geworfen werden.")
}
catch(e: AltersgrenzeUnterschritten){
}

val k3 = spielbank.registriereKunde("Micheal Ballack", 35, 50.00)
val k4 = spielbank.registriereKunde("Steffi Graf", 42, 100.00)

//Fehler KeinAutomatFrei
try{
    val k5 = spielbank.registriereKunde("Angela Merkel", 53, 70.00)
    throw UnsupportedOperationException("Es sollte der Fehler KeinAutomatFrei geworfen werden.")
}
catch(e: KeinAutomatFrei){
}

//entferneKunde()
if (!spielbank.entferneKunde(k3.mitgliedsnummer))
    throw AssertionError("spielbank.entferneKunde(k3.mitgliedsnummer): false != true")
if (spielbank.entferneKunde(k3.mitgliedsnummer))
    throw AssertionError("spielbank.entferneKunde(k3.mitgliedsnummer): true != false")

//registriereMitarbeiter()
val m1 = spielbank.registriereMitarbeiter("Hausmeister Krause", 43)
if (m1.mitarbeiternummer != 2)
    throw AssertionError("spielbank.registriereMitarbeiter(Hausmeister Krause, 43).mitarbeiternummer: ${m1.mitarbeiternummer} != 2")

//entferneKunde()
if (!spielbank.entferneMitarbeiter(m1.mitarbeiternummer))
    throw AssertionError("spielbank.entferneMitarbeiter(m1.mitarbeiternummer): false != true")
if (spielbank.entferneMitarbeiter(m1.mitarbeiternummer))
    throw AssertionError("spielbank.entferneMitarbeiter(m1.mitarbeiternummer): true != false")